In [ ]:
# """
# This script performs the following tasks:
# 1. Authenticates and creates a Google Drive client to interact with Google Drive.
# 2. Downloads all CSV and XLSM files from a specified Google Drive folder to a local destination path.
# 3. Checks if the files exist at the destination path.
# 4. If the files exist:
#     - Reads the files into pandas DataFrames.
#     - Performs a data transformation by converting all column names to lowercase.
#     - Saves the transformed DataFrames to new CSV files.
# 5. If any file does not exist:
#     - Sends an email notification indicating that the file was not found.
# Modules:
# - os: Provides a way of using operating system dependent functionality.
# - smtplib: Defines an SMTP client session object that can be used to send mail.
# - email.mime.text: Defines the MIMEText class, which represents the email message.
# - google.oauth2: Provides OAuth 2.0 authorization for Google APIs.
# - googleapiclient.discovery: Provides access to Google APIs.
# - pandas: Provides data structures and data analysis tools.
# Usage:
# - Ensure that the Google Drive folder ID and destination path are correctly specified.
# - Update the email credentials and SMTP server details for sending notifications.
# """

In [1]:
import os
import smtplib
from email.mime.text import MIMEText

import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
from google.auth.transport.requests import Request

import pandas as pd
import numpy as np

In [2]:
# create client to interact with Google Drive
# Define the scope and authenticate using the service account file
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'credentials.json'
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build the Google Drive service
service = build('drive', 'v3', credentials=credentials)

# google folder id 
FOLDER_ID = "1CpTjXk53N_9-i3dEFjn2Lr17_jYFW-dM?lfhs=2"
LOCAL_FOLDER = "raw_data" 

In [3]:
credentials.refresh(Request())
print("successfully re-authenticated")

successfully re-authenticated


In [4]:
# 先至GCP API & Service 啟用 Google Drive API

def check_api_work():
    try:
        results = service.files().list(fields="files(id, name, mimeType)").execute()
        files = results.get("files", [])

        if not files:
            print("No files found.")
        else:
            print("Files found below:")
            for file in files:
                print(f"{file['name']} {file['mimeType']}")
        return True
    
    except Exception as e:
        print("fail to connect, error message: ", str(e))

        return False

check_api_work()
    

Files found below:
deduction_rate application/vnd.google-apps.spreadsheet
sp01_workdir application/vnd.google-apps.folder
daily_stats.xlsx application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
op_stats application/vnd.google-apps.folder
test1 application/vnd.google-apps.spreadsheet
raw data folder application/vnd.google-apps.folder
int_channel_performance_detail application/vnd.google-apps.spreadsheet
daily_report_dataset_for_etl_test application/vnd.google-apps.spreadsheet
sp01_tb_new_user_recharge.csv text/csv
new_user_recharge_df_trans.xlsx application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
sp01_tb_channel.csv text/csv
ch_df_trans.xlsx application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
sp01_tb_membership.csv text/csv
membership_df_trans.xlsx application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
sp01_tb_recharge.csv text/csv
sp01_tb_op.csv text/csv
op_df_trans.xlsx application/vnd.openxmlformats-officedocument.spreadsheetml.sheet


True

In [5]:
if not os.path.exists(LOCAL_FOLDER):
    os.makedirs(LOCAL_FOLDER)
    print("Filedir not found, create new one")
else:
    print("Filedir found, continue to download files")

Filedir not found, create new one


In [6]:
# 列出 Google Drive 資料夾內的所有檔案與子資料夾
def list_files_and_folders(FOLDER_ID):
    query = f"'{FOLDER_ID}' in parents and trashed=false"
    results = service.files().list(q=query, fields="files(id, name, mimeType)",
                                       supportsAllDrives=True,  # 確保可讀取共享雲端硬碟
                                       includeItemsFromAllDrives=True).execute()
    files = results.get('files', [])
    return files



In [7]:
for file in files:
    print(f"{file['name']} {file['mimeType']}")

NameError: name 'files' is not defined

In [8]:
list_files_and_folders(FOLDER_ID)

HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files?q=%271CpTjXk53N_9-i3dEFjn2Lr17_jYFW-dM%3Flfhs%3D2%27+in+parents+and+trashed%3Dfalse&fields=files%28id%2C+name%2C+mimeType%29&supportsAllDrives=true&includeItemsFromAllDrives=true&alt=json returned "File not found: .". Details: "[{'message': 'File not found: .', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">

In [ ]:
# Function to list files in a Google Drive folder
def list_files_in_folder(folder_id):
    query = f"'{folder_id}' in parents and (mimeType='application/vnd.google-apps.spreadsheet' or mimeType='text/csv')"
    results = service.files().list(q=query, fields='files(id, name)').execute()
    items = results.get('files', [])
    return items

# Function to download a file from Google Drive
def download_file(file_id, destination):
    request = service.files().get_media(fileId=file_id)
    with open(destination, 'wb') as f:
        downloader = MediaIoBaseDownload(f, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f'Download {int(status.progress() * 100)}%')
    return True

In [ ]:
# Function to send email notification
def send_email(subject, body, to_email):
    from_email = 'your_email@example.com'
    password = 'your_password'
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    with smtplib.SMTP('smtp.example.com', 587) as server:
        server.starttls()
        server.login(from_email, password)
        server.sendmail(from_email, to_email, msg.as_string())

# Main execution
def main():
    folder_id = 'your_folder_id_here'  # Replace with your folder ID
    destination_folder = 'path/to/local/folder'  # Replace with your local path
    
    # Create destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)
    
    # Get list of files
    files = list_files_in_folder(folder_id)
    if not files:
        print("No files found in the specified folder")
        send_email('File Not Found', 'No files found in the specified folder', 'recipient@example.com')
        return
    
    # Download each file
    for file in files:
        file_name = file['name']
        file_path = os.path.join(destination_folder, file_name)
        print(f"Downloading {file_name}...")
        
        if download_file(file['id'], file_path):
            print(f"Successfully downloaded {file_name}")
            
            # Process the file based on its type
            try:
                if file_name.endswith('.csv'):
                    df = pd.read_csv(file_path)
                elif file_name.endswith(('.xlsx', '.xlsm')):
                    df = pd.read_excel(file_path)
                
                # Transform column names
                df.columns = df.columns.str.lower()
                
                # Save transformed file
                transformed_path = os.path.join(destination_folder, f'transformed_{file_name}')
                df.to_csv(transformed_path, index=False)
                print(f"Transformed and saved: {transformed_path}")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
        else:
            print(f"Failed to download {file_name}")

# Run the main function
if __name__ == '__main__':
    main()